# 04 - Pitcher Model Training

Train models to predict pitcher fantasy points per IP (skill-based).

**Models:** Random Forest, XGBoost, LightGBM (tree-based only to capture non-linear relationships)

**Validation:** Hold out 2024-2025 data for final evaluation

**Target:** `Fpoints_IP` (fantasy points per inning pitched)

**Note:** This model predicts skill-based points only (3*IP + K - BB - H - 2*ER). W/L/Hold/Save will be added via external projections at prediction time.

In [ ]:
import sys
import os

# Set working directory to project root
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
if 'notebooks' in os.getcwd():
    os.chdir('..')
project_root = os.getcwd()
sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from config.settings import PROCESSED_DATA_DIR, MODELS_DIR, RANDOM_STATE

import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(RANDOM_STATE)

print(f"Project root: {project_root}")

## Load Data

In [ ]:
# Load processed data
df = pd.read_csv(f"{PROCESSED_DATA_DIR}/pitchers_processed.csv")
print(f"Loaded {len(df)} pitcher-seasons")
print(f"Years: {df['Season'].min()} - {df['Season'].max()}")
print(f"\nTarget (Fpoints_IP): mean={df['Fpoints_IP'].mean():.3f}, std={df['Fpoints_IP'].std():.3f}")
print(f"\nRole distribution:")
print(f"  SP (GS > 0): {(df['GS'] > 0).sum()}")
print(f"  RP (GS = 0): {(df['GS'] == 0).sum()}")

In [ ]:
# Identify feature columns (lag and rolling features only)
id_cols = ['IDfg', 'Season', 'Name', 'Team']
target_col = 'Fpoints_IP'

# Model features: lag1, lag2, rolling averages, and arsenal stats
feature_cols = [c for c in df.columns if '_lag' in c or '_avg' in c]

# Also include arsenal columns if present (fastball velo, spin, etc.)
arsenal_cols = [c for c in df.columns if c.startswith(('ff_', 'si_', 'sl_', 'ch_', 'cu_'))]
feature_cols = feature_cols + [c for c in arsenal_cols if c not in feature_cols]

print(f"Number of features: {len(feature_cols)}")
print(f"\nFeature types:")
print(f"  lag1: {len([c for c in feature_cols if '_lag1' in c])}")
print(f"  lag2: {len([c for c in feature_cols if '_lag2' in c])}")
print(f"  rolling avg: {len([c for c in feature_cols if '_avg' in c])}")
print(f"  arsenal: {len(arsenal_cols)}")

## Train/Validation Split

Hold out 2024-2025 for validation (most recent data).

In [ ]:
# Split: train on 2016-2023, validate on 2024-2025
train_years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
val_years = [2024, 2025]

train_df = df[df['Season'].isin(train_years)].copy()
val_df = df[df['Season'].isin(val_years)].copy()

print(f"Training set: {len(train_df)} rows ({train_years[0]}-{train_years[-1]})")
print(f"Validation set: {len(val_df)} rows ({val_years[0]}-{val_years[-1]})")

In [ ]:
# Prepare X and y
X_train = train_df[feature_cols].copy()
y_train = train_df[target_col].copy()

X_val = val_df[feature_cols].copy()
y_val = val_df[target_col].copy()

# Handle any remaining NaN values (fill with column median from training set)
train_medians = X_train.median()
X_train = X_train.fillna(train_medians)
X_val = X_val.fillna(train_medians)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"\nNaN counts - train: {X_train.isna().sum().sum()}, val: {X_val.isna().sum().sum()}")

## Model Training & Hyperparameter Optimization

Tree-based models only (no linear models - we want to capture non-linear relationships).

### 1. Random Forest

In [ ]:
# XGBoost with randomized search
xgb_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5]
}

xgb = XGBRegressor(random_state=RANDOM_STATE, n_jobs=-1, verbosity=0)
xgb_cv = RandomizedSearchCV(xgb, xgb_params, n_iter=10, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1, random_state=RANDOM_STATE)
xgb_cv.fit(X_train, y_train)

print(f"\nBest XGB params: {xgb_cv.best_params_}")
print(f"CV MAE: {-xgb_cv.best_score_:.4f}")

xgb_best = xgb_cv.best_estimator_

### 3. LightGBM

In [ ]:
# LightGBM with randomized search
lgb_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, -1],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [20, 31, 50],
    'subsample': [0.7, 0.8, 1.0]
}

lgb = LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)
lgb_cv = RandomizedSearchCV(lgb, lgb_params, n_iter=10, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1, random_state=RANDOM_STATE)
lgb_cv.fit(X_train, y_train)

print(f"\nBest LGB params: {lgb_cv.best_params_}")
print(f"CV MAE: {-lgb_cv.best_score_:.4f}")

lgb_best = lgb_cv.best_estimator_

### 4. LightGBM

In [ ]:
def evaluate_model(model, X, y, model_name):
    """Evaluate model and return metrics."""
    y_pred = model.predict(X)
    
    mae = mean_absolute_error(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)
    
    return {'Model': model_name, 'MAE': mae, 'RMSE': rmse, 'R²': r2}

# Evaluate all models
results = []
results.append(evaluate_model(rf_best, X_val, y_val, 'Random Forest'))
results.append(evaluate_model(xgb_best, X_val, y_val, 'XGBoost'))
results.append(evaluate_model(lgb_best, X_val, y_val, 'LightGBM'))

results_df = pd.DataFrame(results)
print("\n=== Validation Results ===")
print(results_df.to_string(index=False))

# Select best model based on MAE
best_model_name = results_df.loc[results_df['MAE'].idxmin(), 'Model']
print(f"\nBest model: {best_model_name}")

# Get the best model object
models = {
    'Random Forest': rf_best,
    'XGBoost': xgb_best,
    'LightGBM': lgb_best
}

best_model = models[best_model_name]
needs_scaling = False  # Tree models don't need scaling

In [ ]:
def evaluate_model(model, X, y, model_name, use_scaled=False):
    """Evaluate model and return metrics."""
    if use_scaled:
        y_pred = model.predict(X_val_scaled)
    else:
        y_pred = model.predict(X)
    
    mae = mean_absolute_error(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)
    
    return {'Model': model_name, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

# Evaluate all models
results = []
results.append(evaluate_model(ridge_best, X_val_scaled, y_val, 'Ridge', use_scaled=True))
results.append(evaluate_model(rf_best, X_val, y_val, 'Random Forest'))
results.append(evaluate_model(xgb_best, X_val, y_val, 'XGBoost'))
results.append(evaluate_model(lgb_best, X_val, y_val, 'LightGBM'))

results_df = pd.DataFrame(results)
print("\n=== Validation Results (2024-2025) ===")
print(results_df.to_string(index=False))

In [ ]:
# Get feature importance from tree-based model
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 20 Most Important Features:")
print(importance_df.head(20).to_string(index=False))

## Feature Importance

In [ ]:
# Get feature importance from best tree-based model
if best_model_name in ['Random Forest', 'XGBoost', 'LightGBM']:
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
else:
    # For linear models, use coefficients
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': np.abs(best_model.coef_)
    }).sort_values('importance', ascending=False)

print("Top 20 Most Important Features:")
print(importance_df.head(20).to_string(index=False))

In [ ]:
# Calculate SHAP values for tree-based model
explainer = shap.TreeExplainer(best_model)
shap_values = explainer(X_val)
# Ensure feature names are attached
shap_values.feature_names = feature_cols
print(f"SHAP values calculated for {len(X_val)} validation samples")

## SHAP Analysis

In [ ]:
# Calculate SHAP values for tree-based models
if best_model_name in ['Random Forest', 'XGBoost', 'LightGBM']:
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer(X_val)
    # Ensure feature names are attached
    shap_values.feature_names = feature_cols
    print(f"SHAP values calculated for {len(X_val)} validation samples")
else:
    # For linear models, use Linear explainer
    explainer = shap.LinearExplainer(best_model, X_val_scaled)
    shap_values = explainer(X_val_scaled)
    shap_values.feature_names = feature_cols
    print(f"SHAP values calculated for {len(X_val)} validation samples")

In [ ]:
# SHAP summary plot
plt.figure(figsize=(10, 10))
shap.summary_plot(shap_values, X_val, feature_names=feature_cols, max_display=20, show=False)
plt.tight_layout()
plt.show()

## Player Prediction Function

In [ ]:
def predict_player(player_name, season, show_shap=True):
    """
    Predict fantasy points for a specific pitcher and season.
    Shows SHAP waterfall plot for explainability.
    
    Note: This predicts skill-based Fpoints/IP only.
    W/L/Hold/Save must be added separately from external projections.
    
    Args:
        player_name: Player name (partial match supported)
        season: Season year (e.g., 2024)
        show_shap: Whether to display SHAP waterfall plot
    
    Returns:
        dict with prediction details
    """
    # Find player
    mask = df['Name'].str.contains(player_name, case=False) & (df['Season'] == season)
    player_df = df[mask]
    
    if len(player_df) == 0:
        print(f"Pitcher '{player_name}' not found in {season}")
        # Show similar names
        similar = df[df['Name'].str.contains(player_name, case=False)]['Name'].unique()
        if len(similar) > 0:
            print(f"Similar names: {similar[:5]}")
        return None
    
    if len(player_df) > 1:
        print(f"Multiple matches found: {player_df['Name'].values}")
        player_df = player_df.iloc[[0]]
    
    player_row = player_df.iloc[0]
    X_player = player_df[feature_cols].fillna(train_medians)
    
    # Predict
    if needs_scaling:
        X_player_scaled = scaler.transform(X_player)
        pred = best_model.predict(X_player_scaled)[0]
    else:
        pred = best_model.predict(X_player)[0]
    
    actual = player_row['Fpoints_IP']
    error = pred - actual
    
    # Determine role
    role = "SP" if player_row['GS'] > 0 else "RP"
    
    print(f"\n=== {player_row['Name']} ({season}) ===")
    print(f"Team: {player_row['Team']} | Role: {role}")
    print(f"IP: {player_row['IP']:.1f} | G: {player_row['G']:.0f} | GS: {player_row['GS']:.0f}")
    print(f"\nActual Fpoints/IP (skill): {actual:.3f}")
    print(f"Predicted Fpoints/IP (skill): {pred:.3f}")
    print(f"Error: {error:+.3f}")
    
    # Project total skill-based points (using actual IP)
    total_actual = actual * player_row['IP']
    total_pred = pred * player_row['IP']
    print(f"\nTotal Skill Fpoints (actual IP):")
    print(f"  Actual: {total_actual:.0f}")
    print(f"  Predicted: {total_pred:.0f}")
    
    # Show actual W/L/S/HLD for context
    if 'W' in player_row and 'L' in player_row:
        print(f"\nActual W/L/SV/HLD: {player_row.get('W', 0):.0f}-{player_row.get('L', 0):.0f}, {player_row.get('SV', 0):.0f} SV, {player_row.get('HLD', 0):.0f} HLD")
        # Calculate team-based points
        team_pts = 2*player_row.get('W', 0) - 2*player_row.get('L', 0) + 5*player_row.get('SV', 0) + 2*player_row.get('HLD', 0)
        print(f"  Team-based points: {team_pts:.0f}")
        print(f"  Total actual (skill + team): {total_actual + team_pts:.0f}")
    
    # SHAP waterfall
    if show_shap:
        if needs_scaling:
            player_shap = explainer(X_player_scaled)
        else:
            player_shap = explainer(X_player)
        
        # Attach feature names and UNSCALED data for interpretability
        player_shap.feature_names = feature_cols
        player_shap.data = X_player.values  # Use unscaled values for display
        
        plt.figure(figsize=(10, 8))
        shap.plots.waterfall(player_shap[0], max_display=15, show=False)
        plt.title(f"SHAP Waterfall: {player_row['Name']} ({season})")
        plt.tight_layout()
        plt.show()
    
    return {
        'name': player_row['Name'],
        'season': season,
        'role': role,
        'actual': actual,
        'predicted': pred,
        'error': error,
        'ip': player_row['IP']
    }

In [ ]:
# Example: predict for a specific pitcher (SP)
predict_player("Gerrit Cole", 2024)

In [ ]:
# Try a reliever
predict_player("Emmanuel Clase", 2024)

## 2025 Predictions & Comparison

In [ ]:
def predict_season(season, role_filter=None):
    """
    Predict for all pitchers in a given season and compare to actuals.
    
    Args:
        season: Year to predict
        role_filter: 'SP', 'RP', or None for all
    
    Returns DataFrame with predictions and actuals.
    """
    season_df = df[df['Season'] == season].copy()
    
    if role_filter == 'SP':
        season_df = season_df[season_df['GS'] > 0]
    elif role_filter == 'RP':
        season_df = season_df[season_df['GS'] == 0]
    
    if len(season_df) == 0:
        print(f"No data for {season}")
        return None
    
    X_season = season_df[feature_cols].fillna(train_medians)
    
    if needs_scaling:
        X_season_scaled = scaler.transform(X_season)
        preds = best_model.predict(X_season_scaled)
    else:
        preds = best_model.predict(X_season)
    
    results = season_df[['Name', 'Team', 'IP', 'GS', 'Fpoints_IP']].copy()
    results['Role'] = results['GS'].apply(lambda x: 'SP' if x > 0 else 'RP')
    results['Predicted'] = preds
    results['Error'] = results['Predicted'] - results['Fpoints_IP']
    results['Abs_Error'] = results['Error'].abs()
    
    # Calculate total points (skill-based only)
    results['Actual_Total'] = results['Fpoints_IP'] * results['IP']
    results['Predicted_Total'] = results['Predicted'] * results['IP']
    
    # Sort by predicted total
    results = results.sort_values('Predicted_Total', ascending=False)
    
    return results


def show_season_summary(season, role_filter=None):
    """
    Show summary of predictions for a season.
    
    Args:
        season: Year to show
        role_filter: 'SP', 'RP', or None for all
    """
    results = predict_season(season, role_filter)
    if results is None:
        return
    
    role_str = f" ({role_filter})" if role_filter else ""
    print(f"\n=== {season} Prediction Summary{role_str} ===")
    print(f"Pitchers: {len(results)}")
    print(f"MAE: {results['Abs_Error'].mean():.4f}")
    print(f"RMSE: {np.sqrt((results['Error']**2).mean()):.4f}")
    
    # Correlation between predicted and actual rankings
    results['Actual_Rank'] = results['Actual_Total'].rank(ascending=False)
    results['Predicted_Rank'] = results['Predicted_Total'].rank(ascending=False)
    rank_corr = results['Actual_Rank'].corr(results['Predicted_Rank'])
    print(f"Rank Correlation: {rank_corr:.3f}")
    
    print(f"\nTop 15 Predicted (skill-based Fpoints):")
    display_cols = ['Name', 'Team', 'Role', 'IP', 'Fpoints_IP', 'Predicted', 'Actual_Total', 'Predicted_Total']
    print(results[display_cols].head(15).to_string(index=False))
    
    return results

In [ ]:
# Show 2025 predictions (all pitchers)
results_2025 = show_season_summary(2025)

In [ ]:
# Show 2025 SP only
results_2025_sp = show_season_summary(2025, role_filter='SP')

In [ ]:
# Show 2025 RP only
results_2025_rp = show_season_summary(2025, role_filter='RP')

In [ ]:
# Scatter plot: predicted vs actual
if results_2025 is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # All pitchers
    ax = axes[0]
    colors = results_2025['Role'].map({'SP': 'blue', 'RP': 'orange'})
    ax.scatter(results_2025['Fpoints_IP'], results_2025['Predicted'], c=colors, alpha=0.5)
    min_val = min(results_2025['Fpoints_IP'].min(), results_2025['Predicted'].min())
    max_val = max(results_2025['Fpoints_IP'].max(), results_2025['Predicted'].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect prediction')
    ax.set_xlabel('Actual Fpoints/IP')
    ax.set_ylabel('Predicted Fpoints/IP')
    ax.set_title('2025 Predictions vs Actuals (All)')
    ax.legend(['Perfect', 'SP', 'RP'])
    
    # By role
    ax = axes[1]
    for role, color in [('SP', 'blue'), ('RP', 'orange')]:
        role_df = results_2025[results_2025['Role'] == role]
        ax.scatter(role_df['Fpoints_IP'], role_df['Predicted'], c=color, alpha=0.5, label=role)
    ax.plot([min_val, max_val], [min_val, max_val], 'r--')
    ax.set_xlabel('Actual Fpoints/IP')
    ax.set_ylabel('Predicted Fpoints/IP')
    ax.set_title('2025 Predictions by Role')
    ax.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Show 2024 predictions as well
results_2024 = show_season_summary(2024)

## SP vs RP Model Performance

Check if performance differs significantly between starters and relievers.

In [ ]:
# Compare SP vs RP performance
if results_2025 is not None:
    print("\n=== SP vs RP Model Performance (2025) ===")
    for role in ['SP', 'RP']:
        role_df = results_2025[results_2025['Role'] == role]
        mae = role_df['Abs_Error'].mean()
        rmse = np.sqrt((role_df['Error']**2).mean())
        print(f"{role}: MAE={mae:.4f}, RMSE={rmse:.4f}, n={len(role_df)}")

## Save Best Model

In [ ]:
import os
os.makedirs(MODELS_DIR, exist_ok=True)

# Save model and metadata
model_data = {
    'model': best_model,
    'model_name': best_model_name,
    'scaler': scaler if needs_scaling else None,
    'needs_scaling': needs_scaling,
    'feature_cols': feature_cols,
    'train_medians': train_medians,
    'train_years': train_years,
    'val_results': results_df.to_dict(),
    'note': 'Predicts skill-based Fpoints/IP only. Add W/L/Hold/Save from external projections.'
}

model_path = f"{MODELS_DIR}/pitcher_model.joblib"
joblib.dump(model_data, model_path)
print(f"Model saved to {model_path}")

## Interactive Player Lookup

Use the cells below to look up any pitcher.

In [ ]:
# Change player name and year as needed
predict_player("Zack Wheeler", 2024)

In [ ]:
# List available pitchers for a year
year = 2025
pitchers = df[df['Season'] == year].copy()
pitchers['Role'] = pitchers['GS'].apply(lambda x: 'SP' if x > 0 else 'RP')
pitchers = pitchers.sort_values('Fpoints_IP', ascending=False)

print(f"Top 20 pitchers by Fpoints/IP in {year}:")
print(pitchers[['Name', 'Team', 'Role', 'IP', 'Fpoints_IP']].head(20).to_string(index=False))

In [ ]:
# List top SPs
print(f"\nTop 15 SPs by Fpoints/IP in {year}:")
sps = pitchers[pitchers['Role'] == 'SP']
print(sps[['Name', 'Team', 'IP', 'Fpoints_IP']].head(15).to_string(index=False))

In [ ]:
# List top RPs
print(f"\nTop 15 RPs by Fpoints/IP in {year}:")
rps = pitchers[pitchers['Role'] == 'RP']
print(rps[['Name', 'Team', 'IP', 'Fpoints_IP']].head(15).to_string(index=False))